<br>

# Introdução

In [ ]:
import os
import re
import folium
#import random
#import requests
#import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
#from osgeo import gdal, osr
#from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

In [ ]:
from paths import *

In [ ]:
from open_geodata import lyr

<br>

## Read Data

In [ ]:
gdf = gpd.read_file(
    os.path.join(output_path_geo, 'sp_cetesb.gpkg')
)
gdf.head()
gdf.info()

<br>

## Layer

In [ ]:
def lyr_cetesb():
    # Input
    gdf = gpd.read_file(os.path.join(output_path_geo, 'sp_cetesb.geojson'))
    gdf = gdf.to_crs(epsg=4326)

    # Set dictionary
    colors = {
        'Agência Ambiental de Americana': '#a6cee3',
        'Agência Ambiental de Araraquara': '#1f78b4',
        'Agência Ambiental de Araçatuba': '#b2df8a',
        'Agência Ambiental de Assis': '#33a02c',
        'Agência Ambiental de Atibaia': '#fb9a99',
        'Agência Ambiental de Avaré': '#e31a1c',
        'Agência Ambiental de Barretos': '#4da6ff',
        'Agência Ambiental de Bauru': '#ff7f00',
        'Agência Ambiental de Botucatu': '#cab2d6',
        'Agência Ambiental de Campinas': '#6a3d9a',
        'Agência Ambiental de Capão Bonito': '#ffff99',
        'Agência Ambiental de Cubatão': '#b15928',
        'Agência Ambiental de Dracena': '#a6cee3',
        'Agência Ambiental de Embu das Artes': '#1f78b4',
        'Agência Ambiental de Franca': '#b2df8a',
        'Agência Ambiental de Guarulhos': '#33a02c',
        'Agência Ambiental de Itapetininga': '#fb9a99',
        'Agência Ambiental de Itu': '#e31a1c',
        'Agência Ambiental de Jaboticabal': '#fdbf6f',
        'Agência Ambiental de Jales': '#ff7f00',
        'Agência Ambiental de Jundiaí': '#cab2d6',
        'Agência Ambiental de Limeira': '#6a3d9a',
        'Agência Ambiental de Marília': '#ffff99',
        'Agência Ambiental de Mogi Guaçu': '#b15928',
        'Agência Ambiental de Mogi das Cruzes': '#a6cee3',
        'Agência Ambiental de Osasco': '#00264d',
        'Agência Ambiental de Paulínia': '#b2df8a',
        'Agência Ambiental de Piracicaba': '#33a02c',
        'Agência Ambiental de Presidente Prudente': '#fb9a99',
        'Agência Ambiental de Registro': '#e31a1c',
        'Agência Ambiental de Ribeirão Preto': '#e699ff',
        'Agência Ambiental de Santana': '#ff7f00',
        'Agência Ambiental de Santo Amaro': '#cab2d6',
        'Agência Ambiental de Santos': '#6a3d9a',
        'Agência Ambiental de Sorocaba': '#ffff99',
        'Agência Ambiental de São Carlos': '#ff5050',
        'Agência Ambiental de São José do Rio Preto': '#a6cee3',
        'Agência Ambiental de São José dos Campos': '#1f78b4',
        'Agência Ambiental de São João da Boa Vista': '#b2df8a',
        'Agência Ambiental de São Sebastião': '#33a02c',
        'Agência Ambiental de Taubaté': '#fb9a99',
        'Agência Ambiental de Votuporanga': '#e31a1c',
        'Agência Ambiental do ABC I': '#fdbf6f',
        'Agência Ambiental do ABC II': '#ff7f00',
        'Agência Ambiental do Tatuapé': '#cab2d6',
        'Sede': '#6a3d9a'
    }
    # Popup
    #gdf['PopUp'] = gdf.apply(popup_html, axis=1)
    gdf['popup'] = gdf.apply(popup_html, axis=1)

    # Delete Columns
    gdf.drop([
        'id_municipio', 'endereco', 'numero', 'bairro',
        'municipio_sede', 'cep', 'telefone', 'fax',
        'complemento', 'url', 'email'], axis=1, inplace=True)
    #print(gdf.columns)

    # Layer
    return folium.GeoJson(
        gdf,
        name='CETESB',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        control=True,
        show=False,
        style_function=lambda x: {
            'fillColor': colors[x['properties']['agencia']],
            'color': colors[x['properties']['agencia']],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'agencia'],
            aliases=['Munícipio', 'Agência'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
            # localize=True,
            # style=f"""
            # background-color: #F0EFEF;
            # """,           
        )        
    )
# Add Field
def popup_html(row):
    tel = str(row['telefone']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    fax = str(row['fax']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{}</b></h4></p>
    
    <p><b>Sede:</b><br>
    {}{}{}<br>
    {}
    {}
    {}</p>
    <p><b>Contatos:</b><br>
    {}
    {}
    {}</p>
    <p>{}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['agencia'])        else '{}'.format(row['agencia']),
        '' if pd.isnull(row['endereco'])       else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero'])         else ', {}'.format(row['numero']),
        '' if pd.isnull(row['complemento'])    else ' - {}'.format(row['complemento']),
        '' if pd.isnull(row['bairro'])         else 'Bairro: {}<br>'.format(row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else 'Município: {}<br>'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep'])            else 'CEP: {}<br>'.format(row['cep']),
        '' if pd.isnull(row['telefone'])       else 'Telefone: <a href="tel:{}">{}</a><br>'.format(tel, row['telefone']),
        '' if pd.isnull(row['fax'])            else 'Fax: <a href="tel:{}">{}</a><br>'.format(fax, row['fax']),
        '' if pd.isnull(row['email'])          else 'E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(row['email'])          else '<a href="{}" target="_blank">Conferir Dados</a>'.format(row['url']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

<br>

## Folium

In [ ]:
def get_map(input_geojson):
    # Input
    gdf = gpd.read_file(input_geojson)
    gdf = gdf.to_crs(epsg=4326)
    sw = gdf.bounds[['miny', 'minx']].min().values.tolist()
    ne = gdf.bounds[['maxy', 'maxx']].max().values.tolist()
    bounds = [sw, ne]
    
    # Zoom
    min_zoom = 8
    max_zoom = 21
    padding = 1
    
    # Create Map
    m = folium.Map(
        min_zoom=min_zoom,
        max_zoom=max_zoom,
        max_bounds=True,
        min_lat=bounds[0][0]*((100+padding)/100),
        min_lon=bounds[0][1]*((100+padding)/100),
        max_lat=bounds[1][0]*((100-padding)/100),
        max_lon=bounds[1][1]*((100-padding)/100),
        tiles=None,
        # zoom_delta=0.1,
        # zoom_start=10,
    )
    
    # Add Layers
    m.add_child(lyr.base.google_streets(min_zoom, max_zoom))
    m.add_child(lyr.base.google_terrain(min_zoom, max_zoom))
    m.add_child(lyr.base.google_hybrid(min_zoom, max_zoom))
    m.add_child(lyr.base.google_satellite(min_zoom, max_zoom))

    # Monitoramento
    m.add_child(lyr_cetesb())
    #m.add_child(add_lyr_etes())
    #m.add_child(add_lyr_est_aut_empresas())

    # Bacse PCJ
    #m.add_child(add_lyr_municipios_pcj(base_pcj_path))
    #m.add_child(add_lyr_bacias_pcj(base_pcj_path))    
    #feature_group = folium.FeatureGroup('Hidrografia')
    #feature_group.add_child(add_lyr_hidrografia_principal(base_pcj_path))
    #feature_group.add_child(add_lyr_represas_secundarias(base_pcj_path)) 
    #feature_group.add_child(add_lyr_represas_principais(base_pcj_path))
    #feature_group.add_to(m)

    # Plugins
    m.fit_bounds(bounds)
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    folium.LayerControl(
        position='topright',
        collapsed=False,
    ).add_to(m)
    return m

In [ ]:
# Create Maps
m = get_map(
    os.path.join(output_path_geo, 'sp_cetesb.gpkg')
)

# Save
m.save(os.path.join(output_path_map, 'cetesb_map.html'))
m

<br>

# Get Colors

In [ ]:
def create_colors(input_geojson):
    gdf = gpd.read_file(input_geojson)

    # Column with category
    col_categories = 'agencia'

    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    color_polygon['Agência Ambiental de Ribeirão Preto'] = '#e699ff'
    color_polygon['Agência Ambiental de Barretos'] = '#4da6ff'
    color_polygon['Agência Ambiental de São Carlos'] = '#ff5050'
    color_polygon['Agência Ambiental de Osasco'] = '#00264d'
    return color_polygon

In [ ]:
input_geojson = os.path.join(output_path_geo, 'sp_cetesb.geojson')
a = create_colors(input_geojson)